In [4]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO
import inspect

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
FASTA_PATH = DATA_PATH.joinpath('uniprot-compressed_true_download_true_format_fasta_includeIsoform_tr-2023.02.17-22.00.02.49.fasta')
RESULT_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')
MOTIFS_PATH = DATA_PATH.joinpath('motifs.pkl')
FEATURE_PATH = DATA_PATH.joinpath('mart_export.txt')

In [2]:
df = pd.read_csv(FEATURE_PATH, sep='\t')

df = df.groupby(['Gene Synonym']).filter(lambda x: x['Gene name'].dropna().drop_duplicates().shape[0] == 1)

df = df[['Gene Synonym', 'Gene name']]

df = df.drop_duplicates()

df = df.set_index(['Gene Synonym'])

ds = df['Gene name']

gene_synonym_gene_name = ds.to_dict()

In [3]:
records = []
for data in SeqIO.parse(FASTA_PATH, "fasta"):
    record={}
    for index, value in enumerate(re.findall(r'(?:(?<=^)[^|]+|(?<=\|)[^| ]+|(?<=\|)[^ ]+)|(?<=\ ).+?(?= \w\w\=|$)', data.description)):
        value = value.strip()
        if index == 0:
            record['db'] = value
        elif index == 1:
            record['UniqueIdentifier'] = value
        elif index == 2:
            record['EntryName'] = value
        elif index == 3:
            record['ProteinName'] = value
        else:
            k, v = re.split(r'\=', value)
            record[k] = v

    record['seq'] = str(data.seq)
    
    records.append(record)

In [4]:
df = pd.DataFrame(records)

df['gene_symbol_harmonized'] = df['GN'].apply(lambda x: gene_synonym_gene_name.get(x, x))

df = df.set_index('gene_symbol_harmonized').reset_index()

_df_protein_sequence = df.copy()

In [5]:
df = _df_protein_sequence.copy()

df = df.merge(pd.read_pickle(MOTIFS_PATH), how='outer')

df = df[['gene_symbol_harmonized', 'db', 'UniqueIdentifier', 'EntryName', 'ProteinName', 'OS', 'OX', 'GN', 'PE', 'SV', 'motifs', 'seq']]

assert not df['UniqueIdentifier'].duplicated().any()

df.to_pickle(RESULT_PATH)

In [7]:
df = pd.read_pickle(RESULT_PATH)

df.loc[df['motifs'].notna()]

,gene_symbol_harmonized,db,UniqueIdentifier,EntryName,ProteinName,OS,OX,GN,PE,SV,motifs,seq
13,Arrdc4,sp,A0A0B4J1F4,ARRD4_MOUSE,Arrestin domain-containing protein 4,Mus musculus,10090,Arrdc4,1,1,PPNY PPLY,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEPVALRGLRLEAQGRATSAWGPSAGARVCIGGGSPAASSEVEYLNLRLSLLE...
19,Eif4a3l1,tr,A0A0N4SVM5,A0A0N4SVM5_MOUSE,RNA helicase (Fragment),Mus musculus,10090,Eif4a3l1,4,1,PTFDTMGLREDLLRGIYAYGFEKPSAIQQ,MATTTRMATSGSARKRLLKEDGMTKVEFETSEEVDATPTFDTMGLREDLLRGIYAYGFEKPSAIQQRAIKQIMKGRDVIAQSQSGTGKTATFSISV...
20,Eif4a3l2,tr,A0A0N4SVP8,A0A0N4SVP8_MOUSE,RNA helicase,Mus musculus,10090,Eif4a3l2,3,1,LTFDTMGLREDLLRGIYAYGFEKPSAIQQ,MATTARMATSVSARKRLLKEEGMTKVEFETSEEVDRTLTFDTMGLREDLLRGIYAYGFEKPSAIQQRAIKQIIKGRDVIAQSQSGTGKTATFSISV...
50,Gcna,sp,A0A1D9BZF0,GCNA_MOUSE,Germ cell nuclear acidic protein,Mus musculus,10090,Gcna,1,1,SGSS CVVI VCEI LIVI,MDSGSSSSSSSSGSSSGSCSTSGSGSTSGSSTTSSSSSSSSSSSSSSSSSSKEYMPELPKQRKASCVVIDSESDSDNTSDEKNTTVCEISSGDETS...
55,Nova2,sp,A0A1W2P872,NOVA2_MOUSE,RNA-binding protein Nova-2,Mus musculus,10090,Nova2,1,1,KRPLETPPEVVCTKRSN,MEPEAPDSRKRPLETPPEVVCTKRSNTGEEGEYFLKVLIPSYAAGSIIGKGGQTIVQLQKETGATIKLSKSKDFYPGTTERVCLVQGTAEALNAVH...
60,Spast,tr,A0A286YE25,A0A286YE25_MOUSE,Spastin,Mus musculus,10090,Spast,1,1,PAGRRKKK VGFALLRLL RKKK,MSSPAGRRKKKGSGGASPAPARPPPPAAVPAPAAGPAPAAGSPPKRNPSSFSSPLVVGFALLRLLACHLGLLFAWLCQRFSRALMAAKRSSGTAPA...
67,Spast,tr,A0A3B2WC92,A0A3B2WC92_MOUSE,Spastin,Mus musculus,10090,Spast,1,1,PAGRRKKK VGFALLRLL RKKK,MSSPAGRRKKKGSGGASPAPARPPPPAAVPAPAAGPAPAAGSPPKRNPSSFSSPLVVGFALLRLLACHLGLLFAWLCQRFSRALMAAKRSSGTAPA...
99,Cul4b,sp,A2A432,CUL4B_MOUSE,Cullin-4B,Mus musculus,10090,Cul4b,1,1,KKRK,MSRSTRSKERRENDTDSEDNSSETSNQERRRCRQGPPRPPYPPLLPPVFPPPTPPPQVRRTRGLQDLGAMKSVCPGTSGFSSPNPSAASAAAQEVR...
104,Sppl2c,sp,A2A6C4,SPP2C_MOUSE,Signal peptide peptidase-like 2C,Mus musculus,10090,Sppl2c,1,1,PAL,MACLGSLHPLGSLLLLFLLLLLSPEARGEYGLVRVVSKNWSKDYCVLYSSDYVNLPRDLHHAPLLSLHDGTKTPWCPDEDSFHQAQDSSPRQRPLH...
105,Calcoco2,sp,A2A6M5,CACO2_MOUSE,Calcium-binding and coiled-coil domain-containing protein 2,Mus musculus,10090,Calcoco2,2,1,IMVV ASWE,MDQCPIPTLLEHGNFSQVLFNNVEKFYAPRGDIMCYYTLTEKFIPRRKDWIGIFKVGWKTTQEYYTFMWAPLPKDQNKDSATQQEIQFKAYYLPKD...
